In [15]:
%reset

In [1]:
import psycopg2
import pandas as pd
import numpy as np

In [6]:
"""коннект"""
con=psycopg2.connect(
    host="localhost",
    port=5432,
    database="movie_db",
    user="postgres",
    password="1",
)

querry_comm="""select * from table_name"""
communications=pd.read_sql(querry_comm,con)
communications.head(5)

""


In [18]:
"""запрос к communications таблице"""
querry_comm="""select * from communications"""
communications=pd.read_sql(querry_comm,con)
communications.head(5)

,communication_id,site_id,visitor_id,date_time
0,460453730,16649,615231236,2018-04-05 17:35:16
1,2092162,16649,658040990,2018-04-07 16:48:44
2,2061609,16649,848021975,2018-04-02 09:20:20
3,458670105,30374,1225592954,2018-04-02 18:38:03
4,2092238,16649,1235856228,2018-04-07 17:09:21


In [19]:
"""запрос к sessions таблице"""
querry_session="""select * from sessions"""
sessions=pd.read_sql(querry_session,con)
sessions.head(5)

,visitor_session_id,site_id,visitor_id,date_time,campaign_id
0,2038323227,16649,1322630246,2018-04-03 22:39:12,44843
1,2043591207,16649,1325892110,2018-04-04 19:45:39,116083
2,2026999821,16649,1315616600,2018-04-02 08:56:25,116083
3,2038059953,30374,1322459808,2018-04-03 21:38:44,116085
4,2030418799,16649,1317685222,2018-04-02 17:02:44,116083


In [20]:
"""создание новых колонок с количеством сессий и датой от последней коммуникации"""
%time
session_date_time_list=[]
row_n_list=[]
for i in range(len(communications['visitor_id'])):
    try:
        session_date_time_list.append(sessions[sessions['visitor_id']==communications['visitor_id'][i]]['date_time'][sessions['date_time']<=
        communications['date_time'][communications['visitor_id']==communications['visitor_id'][i]].iloc[0]].sort_values(ascending=False).iloc[1])
        row_n_list.append(len(sessions[sessions['visitor_id']==communications['visitor_id'][i]]['date_time'][sessions['date_time']<=
        communications['date_time'][communications['visitor_id']==communications['visitor_id'][i]].iloc[0]].sort_values(ascending=False).iloc[1:]))
    except IndexError:
        session_date_time_list.append(np.NaN)
        row_n_list.append(0)

Wall time: 0 ns


In [21]:
joined=communications.merge(sessions,how='left',on='visitor_id').\
    drop_duplicates(subset=['communication_id']).reset_index(drop=True)

joined['row_n']=row_n_list
joined['sessions_date_time']=session_date_time_list

In [76]:
joined.head(5)

,communication_id,site_id_x,visitor_id,date_time_x,visitor_session_id,site_id_y,date_time_y,campaign_id,row_n,sessions_date_time
0,460453730,16649,615231236,2018-04-05 17:35:16,2048740059,16649,2018-04-05 17:31:37,116083,0,NaT
1,2092162,16649,658040990,2018-04-07 16:48:44,2054754818,16649,2018-04-07 16:38:21,-1,7,2018-04-07 12:23:22
2,2061609,16649,848021975,2018-04-02 09:20:20,2027105873,16649,2018-04-02 09:15:08,116083,0,NaT
3,458670105,30374,1225592954,2018-04-02 18:38:03,2030992565,30374,2018-04-02 18:37:12,116085,0,NaT
4,2092238,16649,1235856228,2018-04-07 17:09:21,2054740380,16649,2018-04-07 16:32:30,116083,0,NaT


In [8]:
"""выгрзука в таблицу"""
from sqlalchemy import create_engine

#выгрузка
# 'postgresql://postgres:1@localhost/users'
engine = create_engine('postgresql://postgres:1@localhost/movie_db', echo=False)

# joined.to_sql('joined', con=engine)
pd.read_sql("""select * from table_name""", engine)

""


In [29]:
pd.read_sql("""select * from joined""", con)

,index,communication_id,site_id_x,visitor_id,date_time_x,visitor_session_id,site_id_y,date_time_y,campaign_id,row_n,sessions_date_time
0,0,460453730,16649,615231236,2018-04-05 17:35:16,2048740059,16649,2018-04-05 17:31:37,116083,0,NaT
1,1,2092162,16649,658040990,2018-04-07 16:48:44,2054754818,16649,2018-04-07 16:38:21,-1,7,2018-04-07 12:23:22
2,2,2061609,16649,848021975,2018-04-02 09:20:20,2027105873,16649,2018-04-02 09:15:08,116083,0,NaT
3,3,458670105,30374,1225592954,2018-04-02 18:38:03,2030992565,30374,2018-04-02 18:37:12,116085,0,NaT
4,4,2092238,16649,1235856228,2018-04-07 17:09:21,2054740380,16649,2018-04-07 16:32:30,116083,0,NaT
5,5,2092428,16649,1279156408,2018-04-07 18:21:16,2054991378,16649,2018-04-07 18:19:41,116083,0,NaT
6,6,457537963,16649,1310647296,2018-04-01 10:00:28,2022247367,16649,2018-04-01 09:44:57,116083,0,NaT
7,7,456995966,16649,1310647296,2018-04-01 10:16:50,2022247367,16649,2018-04-01 09:44:57,116083,0,NaT
8,8,2056235,16649,1311935382,2018-04-01 00:21:49,2021383539,16649,2018-04-01 00:18:21,116083,0,NaT
9,9,2056231,16649,1311935382,2018-04-01 00:21:07,2021383539,16649,2018-04-01 00:18:21,116083,0,NaT


In [374]:
"""скрипт который создается в постгрес(задание postgres) """
sql_script="""

create table from_py_sql as
(
    SELECT FRT.visitor_id,communication_id,site_id,row_n,campaign_id,FRT.sessions_date_time,date_time as communication_date_time from (
                    SELECT * from communications com
                    left join LATERAL
                    (
                        select LEAD(date_time) over (order by date_time desc) as sessions_date_time, visitor_session_id,campaign_id
                        from sessions
                        where (visitor_id = com.visitor_id)
                        and (site_id = com.site_id)
                        and (date_time<=com.date_time)
                        limit 1
                    ) re on com.visitor_id = visitor_id
                  ) FRT
    left join LATERAL (
                        select Count(date_time)-1 as row_n, visitor_id
                        from LATERAL (
                                        select date_time
                                        from sessions
                                        where (visitor_id = FRT.visitor_id)
                                        and (date_time <=frt.date_time)
                                     ) a where visitor_id = FRT.visitor_id and site_id = frt.site_id
                      ) sss
    on FRT.visitor_id = sss.visitor_id

)
"""


In [381]:
"""создаем таблицу в бд """
curs=con.cursor()


curs.execute(sql_script)
con.commit() # <--- makes sure the change is shown in the database
con.close()
curs.close()